In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

# Loading data and preprocessing

In [170]:
data=sc.textFile("C:/Users/Admin/Downloads/project.csv")

In [33]:
##type of data
type(data)

pyspark.rdd.RDD

In [57]:
##length of the data
data.count()

45212

In [78]:
##first 5 rows of the data
data.take(5)

['"age,""job"",""marital"",""education"",""default"",""balance"",""housing"",""loan"",""contact"",""day"",""month"",""duration"",""campaign"",""pdays"",""previous"",""poutcome"",""y"""',
 '"58,""management"",""married"",""tertiary"",""no"",2143,""yes"",""no"",""unknown"",5,""may"",261,1,-1,0,""unknown"",""no"""',
 '"44,""technician"",""single"",""secondary"",""no"",29,""yes"",""no"",""unknown"",5,""may"",151,1,-1,0,""unknown"",""no"""',
 '"33,""entrepreneur"",""married"",""secondary"",""no"",2,""yes"",""yes"",""unknown"",5,""may"",76,1,-1,0,""unknown"",""no"""',
 '"47,""blue-collar"",""married"",""unknown"",""no"",1506,""yes"",""no"",""unknown"",5,""may"",92,1,-1,0,""unknown"",""no"""']

### Transformation of the data

In [171]:
#We may need to note that each row of the data is a character string, and it would be more convenient to have an array 
#in some senarios. We can use map to transform them
content = data.map(lambda x: x.split(','))
content.take(3)

[['"58',
  '""management""',
  '""married""',
  '""tertiary""',
  '""no""',
  '2143',
  '""yes""',
  '""no""',
  '""unknown""',
  '5',
  '""may""',
  '261',
  '1',
  '-1',
  '0',
  '""unknown""',
  '""no"""'],
 ['"44',
  '""technician""',
  '""single""',
  '""secondary""',
  '""no""',
  '29',
  '""yes""',
  '""no""',
  '""unknown""',
  '5',
  '""may""',
  '151',
  '1',
  '-1',
  '0',
  '""unknown""',
  '""no"""'],
 ['"33',
  '""entrepreneur""',
  '""married""',
  '""secondary""',
  '""no""',
  '2',
  '""yes""',
  '""yes""',
  '""unknown""',
  '5',
  '""may""',
  '76',
  '1',
  '-1',
  '0',
  '""unknown""',
  '""no"""']]

In [172]:
## let us replace the "" by ''
def clean(x):
        return([xx.replace('""', '').replace('"','') for xx in x])

content = content.map(clean)

A=content.take(3)

In [169]:
df.count()


45212

In [173]:
##transformation of data into pyspark dataframe

dat = content.map(lambda x: Row(age=x[0], job=x[1],marital=x[2],education=x[3],default=x[4],balance=x[5],housing=x[6],
                         loan=x[7],contact=x[8],day=x[9],month=x[10],duration=x[11],campaign=x[12],         
                         pdays=x[13],previous=x[14],poutcome=x[15],y=x[16]))
df = sqlContext.createDataFrame(dat)

In [210]:
#let us view the first two rows
df.show(2,truncate= True)

+---+-------+--------+-------+---+-------+--------+---------+-------+----------+----+-------+-----+-----+--------+--------+---+
|age|balance|campaign|contact|day|default|duration|education|housing|job       |loan|marital|month|pdays|poutcome|previous|y  |
+---+-------+--------+-------+---+-------+--------+---------+-------+----------+----+-------+-----+-----+--------+--------+---+
|58 |2143   |1       |unknown|5  |no     |261     |tertiary |yes    |management|no  |married|may  |-1   |unknown |0       |no |
|44 |29     |1       |unknown|5  |no     |151     |secondary|yes    |technician|no  |single |may  |-1   |unknown |0       |no |
+---+-------+--------+-------+---+-------+--------+---------+-------+----------+----+-------+-----+-----+--------+--------+---+
only showing top 2 rows



In [141]:
#description of the dataframe
df.describe().show()

+-------+-----------------+------------------+------------------+--------+-----------------+-------+-----------------+---------+-------+-------+-----+--------+-----+------------------+--------+------------------+-----+
|summary|              age|           balance|          campaign| contact|              day|default|         duration|education|housing|    job| loan| marital|month|             pdays|poutcome|          previous|    y|
+-------+-----------------+------------------+------------------+--------+-----------------+-------+-----------------+---------+-------+-------+-----+--------+-----+------------------+--------+------------------+-----+
|  count|            45212|             45212|             45212|   45212|            45212|  45212|            45212|    45212|  45212|  45212|45212|   45212|45212|             45212|   45212|             45212|45212|
|   mean|40.93621021432837|1362.2720576850766| 2.763840658246887|    null|15.80641879188693|   null|258.1630797814691|     n

## 2. Some Statistics

### Marketing success  rate

In [212]:
N=df.count()
df.crosstab('y','y').show()

+---+-----+----+
|y_y|   no| yes|
+---+-----+----+
|yes|    0|5289|
| no|39922|   0|
+---+-----+----+



In [158]:
print('the marketing sucess rate is {} %'.format(round(5289*100/N,2)))
print('the marketing failure rate is {} %'.format(round(39922*100/N,2)))  

the marketing sucess rate is 11.7 %
the marketing failure rate is 88.3 %


### . Give the maximum, mean, and minimum age of the average targeted customer


In [213]:
df.describe('age').show()

+-------+-----------------+
|summary|              age|
+-------+-----------------+
|  count|            45211|
|   mean|40.93621021432837|
| stddev|10.61876204097542|
|    min|               18|
|    max|               95|
+-------+-----------------+



In [176]:
print('the maximum, mean, and the minimum age of the targeted customer are respectively {} ,{} and {}'.format
      (95,40.94,18))

the maximum, mean, and the minimum age of the targeted customer are respectively 95 ,40.94 and 18


### Check the quality of customers by checking average balance, median balance of customers


In [179]:
df.describe('balance').show()


+-------+------------------+
|summary|           balance|
+-------+------------------+
|  count|             45211|
|   mean|1362.2720576850766|
| stddev| 3044.765829168523|
|    min|                -1|
|    max|              9997|
+-------+------------------+



In [247]:
#conversion of string to float
bal=df.select(df.balance.cast("float"))
type(bal)
 

pyspark.sql.dataframe.DataFrame

In [248]:
bal.approxQuantile('balance',probabilities=[0.5],relativeError=1)


[-8019.0]

In [249]:
print("the mean and median of the balance are far differents ")

the mean and median of the balance are far differents 


### Check if age matters in marketing subscription for deposit

In [180]:
df.groupby('y').agg({'age':'mean'}).show()

+---+------------------+
|  y|          avg(age)|
+---+------------------+
| no| 40.83898602274435|
|yes|41.670069956513515|
+---+------------------+



In [251]:
print('The difference between the mean average is of 0.84 year. age matters a litte bit on the subscription')

The difference between the mean average is of 0.84 year. age matters a litte bit


### Check if marital status mattered for a subscription to deposit

In [181]:
df.crosstab('y', 'marital').show()

+---------+--------+-------+------+
|y_marital|divorced|married|single|
+---------+--------+-------+------+
|      yes|     622|   2755|  1912|
|       no|    4585|  24459| 10878|
+---------+--------+-------+------+



In [253]:
#percentage of subscription by marital status
mari=['divorced', 'married', 'single']
A=[622,2755,1912]; B=[5207,27214,12790]
C=[A[i]*100/B[i] for i in range(3)]

for i in range(3):
    print("percentage of subscription for " +str(mari[i])+" is {}".format(round(C[i],2)))

percentage of subscription for divorced is 11.95
percentage of subscription for married is 10.12
percentage of subscription for single is 14.95


In [254]:
print('we can conclude that there is a effect of the marital status on the susbscription: ')
print('for instance single are more excited to subscribe whereas married are less excited') 

we can conclude that there is a effect of the marital status on the susbscription: 
for instance single are more excited to subscribe whereas married are less excited


### Check if age and marital status together mattered for a subscription to deposit scheme

In [259]:
df.groupby('y','marital').agg({'age':'mean'}).show()

+---+--------+------------------+
|  y| marital|          avg(age)|
+---+--------+------------------+
| no| married| 43.05854695613067|
| no|  single| 33.96258503401361|
|yes|  single| 32.22907949790795|
|yes| married| 46.51143375680581|
| no|divorced| 45.31297709923664|
|yes|divorced|49.247588424437296|
+---+--------+------------------+



In [256]:
print("as in the above table, we can see that there is a join effect of age and marital status on subscription")

as in the above table, we can see that there is a join effect of age and marital status on subscription


### Do feature engineering for the bank and find the right age effect on the campaign.

In [269]:
from pyspark.sql.functions import when

df2=df.withColumn("g_age",when(df.age>65, 'Old').when( df.age>30, 'mid').otherwise('Young'))
df2.show(5)

+---+-------+--------+-------+---+-------+--------+---------+-------+------------+----+-------+-----+-----+--------+--------+---+-----+
|age|balance|campaign|contact|day|default|duration|education|housing|         job|loan|marital|month|pdays|poutcome|previous|  y|g_age|
+---+-------+--------+-------+---+-------+--------+---------+-------+------------+----+-------+-----+-----+--------+--------+---+-----+
| 58|   2143|       1|unknown|  5|     no|     261| tertiary|    yes|  management|  no|married|  may|   -1| unknown|       0| no|  mid|
| 44|     29|       1|unknown|  5|     no|     151|secondary|    yes|  technician|  no| single|  may|   -1| unknown|       0| no|  mid|
| 33|      2|       1|unknown|  5|     no|      76|secondary|    yes|entrepreneur| yes|married|  may|   -1| unknown|       0| no|  mid|
| 47|   1506|       1|unknown|  5|     no|      92|  unknown|    yes| blue-collar|  no|married|  may|   -1| unknown|       0| no|  mid|
| 33|      1|       1|unknown|  5|     no|     1

In [278]:
df2.crosstab('y', 'g_age').show()

+-------+---+-----+-----+
|y_g_age|Old|Young|  mid|
+-------+---+-----+-----+
|    yes|320| 1145| 3824|
|     no|431| 5885|33606|
+-------+---+-----+-----+



In [279]:
#percentage of subscription by gpe age
ag=['Old', 'Young', 'mid']
A=[320,1145,3821]; B=[751,7030,37430]
C=[A[i]*100/B[i] for i in range(3)]

for i in range(3):
    print("percentage of subscription for " +str(ag[i])+" is {}".format(round(C[i],2)))

percentage of subscription for Old is 42.61
percentage of subscription for Young is 16.29
percentage of subscription for mid is 10.21


In [280]:
print("We can then conclude that Old people are much interested meanwhile Mid people are less interested")

We can then conclude that Old people are much interested meanwhile Mid people are less interested


In [ ]:
## important link https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/